# L17: Mutability in Lettuce

## Overview
* Implicit References
* Eval with Implicit References
* CPS Intro

## Implicit References
* We looked at explicit references for mutability in lettuce (like Java and C++)
* it's fine...
* what about implicit references? (like Python)
* Implicit reference provide, what many consider, easier syntax
* They behave a bit differently under the hood (because the syntax is ambiguous)
* Consider the following

### Concrete Lettuce Grammar - implicit refs
$$\begin{array} &
e & \rightarrow & let\ var\ x\ =\ e\ in\ e \\
& | & AssignVar(x,e) \\
\\
x & \rightarrow & identifier \\
\\
\end{array} $$

#### Concrete Lettuce Examples

##### Easy Example
###### EXPLICIT REF
~~~
let x = NewRef(10) in 
let y = DeRef(x) + 1 in 
let z = AssignRef(x, y) in
DeRef(x)
~~~
###### Implicit REF
~~~
let var x = 10 in
let y = x + 1 in
let z = AssignVar(x,y) in
x
~~~

##### Hard Example
###### EXPLICIT REF
~~~
let incr = function (x) AssignRef(x, DeRef(x) + 1)
in 
let double = function (x) AssignRef(x, DeRef(x) * 2)
in 
let r = NewRef(15) in 
let tmp0 = double(r) in 
let tmp1 = incr(r) in 
let tmp2 = double(r) in
DeRef(r) 
~~~
###### Implicit REF
~~~
let incr = function (x) AssignVar(x, x + 1)
in 
let double = function (x) AssignVar(x, x * 2)
in 
let var r = 15 in 
let tmp0 = double(r) in 
let tmp1 = incr(r) in 
let tmp2 = double(r) in
r
~~~

#### Comments on Concrete Lettuce
* you can decide for yourself which is a "better" syntax
* It's an aesthetic choice
* Doesn't make much of a differnce in the performance of the langauge
* It does change how the programmer needs to think while programming
* Implicit reference trade-off:
    * less syntax to remember
        * As a lettuce programmer I don't need to type DeRef and NewRef...
    * less explicit syntax, ambiguous syntax
        * the value of identifier 'x' might be a reference to something in our mememory/ImmutableStore
            * but we likely would return the value in memory rather than the reference itself
        * the value of identifier 'x' might be something simple and exists in the environment

### Generative Grammar for Lettuce AST - implicit refs
$$\begin{array} &
Expr & \rightarrow & LetVar(Identifier,Expr,Expr)\\
& | & AssignVar(Identifier,Expr)\\
& | & Value \\
\\
Value & \rightarrow & Reference(Address) \\
\\
Identifier & \rightarrow & String \\
\\
Address & \rightarrow & Int \\
\\
\end{array} $$

* NOTE: AssignVar encapsulates an Identifier and an Expr

### Visualizing memory
* For our purposes memory will be an immutable store
    * We can create a new stores as we go along
    * These new stores will look like the old store, but with incremental changes
* Let us visualize this as a table that grows
    * In real life, this is a table of fixed size and free space that is populated somewhat randomly over time
    * There is nothing truely random about that - usually
    * But the determinism of how space is allocated is quite complex and hardware dependent
    * So we won't worry about it in this course
    * Computer Engineering, OS, and Compilers would teach you more about this topic
* I begin with a empty table:
<table>
    <caption>Memory Size: 0</caption>
    <tr>
        <th>Address</th>
        <th>Value</th>
    </tr>
</table>

* Suppose I write "let var x = 10 in ..." I:
    * must create a memory store
        * the store is now length 1
        * the store has a new address '0'
        * in that address I put the value 10
    * I can visualize this as follows:
<table>
    <caption>Memory Size: 1</caption>
    <tr>
        <th>Address</th>
        <th>Value</th>
    </tr>
    <tr>
        <td>0</td>
        <td>10</td>
    </tr>
</table>

* Questions?

#### Visualizing memory - easy example
Let us look at how the following expression evaluates:
~~~
let var x = 10 in
let y = x + 1 in
let z = AssignVar(x,y) in
x
~~~

##### 1 - easy
* set up the problem

~~~
EVALUATE:
    let var x = 10 in
    let y = x + 1 in
    let z = AssignVar(x,y) in
    x
IN ENVIRONMENT:
    {}
AND STORE:
~~~
<table>
    <caption>Memory Size: 0</caption>
    <tr>
        <th>Address</th>
        <th>Value</th>
    </tr>
</table>

##### 2 - easy
~~~
let var x = 10 in
let y = x + 1 in
let z = AssignVar(x,y) in
x
~~~
* We have pattern 'let var x = e_binding in e_body'
    * x = x
    * e_binding = 10
    * e_body = let y = x + 1 in [ let z = AssignVar(x,y) in x ]
* Evaluate the e_binding expression to a v_binding
    * 10
    * 10 is a value
* because we have 'var', we must allocate a memory cell for v_binding_ref
    * the length of memory is 0, so in location 0 we put v_binding of 10, or rather, Const(10.0)
    * length of memory is now 1
    * we find Reference(0) for v_binding_ref
* map identifier 'x' to v_binding_ref 'Reference(0)' in the environment
* next we will need to evaluate the body expression 'e_body'
    * let y = x + 1 in [ let z = AssignVar(x,y) in x ]
    
~~~
EVALUATE:
    let y = x + 1 in
    let z = AssignVar(x,y) in
    x
IN ENVIRONMENT:
    {
    x -> Reference(0)
    }
AND STORE:
~~~
<table>
    <caption>Memory Size: 1</caption>
    <tr>
        <th>Address</th>
        <th>Value</th>
    </tr>
    <tr>
        <td>0</td>
        <td>10</td>
    </tr>
</table>

##### 3 - easy
~~~
let y = x + 1 in
let z = AssignVar(x,y) in
x
~~~
* We have pattern 'let x = e_binding in e_body'
    * x = y
    * e_binding = x + 1
    * e_body = let z = AssignVar(x,y) in x
* Evaluate the e_binding expression to a v_binding
    * x + 1
    * pattern: e1 + e2
        * evaluate e1 to value v1
            * x
            * in our environment idenitfier 'x' is mapped to Reference(0)
            * **HERE** is an important bit
                * Reference(0) is a value
                * but I don't want to use it here
                * instead I want to go deeper
                * look in memory/state/store and find what is at Reference(0)
            * 10
        * evaluate e2 to value v2
            * 1
        * calculate v1 + v2
            * 10 + 1
            * 11
* map identifier 'y' to v_binding - Const(11) - in the environment
* next we will need to evaluate the body expression 'e_body'
    * let z = AssignVar(x,y) in x
    
~~~
EVALUATE:
    let z = AssignVar(x,y) in
    x
IN ENVIRONMENT:
    {
    y -> Const(11.0),
    x -> Reference(0)
    }
AND STORE:
~~~
<table>
    <caption>Memory Size: 1</caption>
    <tr>
        <th>Address</th>
        <th>Value</th>
    </tr>
    <tr>
        <td>0</td>
        <td>10</td>
    </tr>
</table>

##### 4 - easy
~~~
let z = AssignVar(x,y) in
x
~~~
* We have pattern 'let x = e_binding in e_body'
    * x = z
    * e_binding = AssignVar(x,y)
    * e_body = x
* Evaluate the e_binding expression to a v_binding
    * AssignVar(x,y)
    * **HERE** is an important bit
        * pattern AssignVar(x, er)
            * x = x
            * er = y
        * find location value 'vl' by lookuping 'x' in the environment
            * Reference(0)
            * if this wasn't a reference, we would have an issue - thow error
        * evaluate er to a value 'vr'
            * y
            * looup y in the environment
            * 11
        * go to location vl and put vr there
            * change the value at address 0 from 10 to 11
        * return vr - 11 - as v_binding
* map identifier 'z' to v_binding - Const(11) - in the environment
* next we will need to evaluate the body expression 'e_body'
    * x
    
~~~
EVALUATE:
    x
IN ENVIRONMENT:
    {
    z -> Const(11.0),
    y -> Const(11.0),
    x -> Reference(0)
    }
AND STORE:
~~~
<table>
    <caption>Memory Size: 1</caption>
    <tr>
        <th>Address</th>
        <th>Value</th>
    </tr>
    <tr>
        <td>0</td>
        <td>11</td>
    </tr>
</table>

##### 5 - easy
~~~
x
~~~
* We have pattern 'x'
    * x = x
* lookup 'x' in or environment
    * x -> Reference(0)
    * **HERE** is the important bit
        * Reference(0) is a value
        * but I don't want to use it here
        * instead I want to go deeper
        * look in memory/state/store and find what is at Reference(0)
    * 11
    
~~~
EVALUATE:
    11
IN ENVIRONMENT:
    {
    z -> Const(11.0),
    y -> Const(11.0),
    x -> Reference(0)
    }
AND STORE:
~~~
<table>
    <caption>Memory Size: 1</caption>
    <tr>
        <th>Address</th>
        <th>Value</th>
    </tr>
    <tr>
        <td>0</td>
        <td>11</td>
    </tr>
</table>

##### 6 - easy
~~~
11
~~~
* 11 is a value
* We're done!

#### Visualizing memory - observations
* We are doing something special when we look at pattern 'x', or rather AST nodes of the form Ident(x:Identifier)
* There is also something interesting about AssignVar(x:Identifier, e:Expr)

### Generative Grammar for Lettuce AST - implicit refs
$$\begin{array} &
Expr & \rightarrow & LetVar(Identifier,Expr,Expr)\\
& | & AssignVar(Identifier,Expr)\\
& | & Value \\
\\
Value & \rightarrow & Reference(Address) \\
\\
Identifier & \rightarrow & String \\
\\
Address & \rightarrow & Int \\
\\
\end{array} $$

## Eval with Implicit References

### Inference Rules
* It might be nice to see some of the inference rules here
* Operational Semantic:
    * (v, mp) = eval(e, env, m)
* semantic variables
    * e is an expression
    * v is a value (a subset of expressions)
    * env is an environment
    * m is a memory store
    * a is an address (a subset of values)
    * x is an identifier
    * p denotes a priming of any of the above
* other operational semantics used
    * v = lookupEnv(env, x)
    * v = lookupCellValue(m, a)
    * mp = assignToCell(m, a, v)
    * (mp, a) = createNewCell(m, v)
    * e $\neq$ e

#### Ident Inference Rules
<br />
$$\begin{array} &
a\ =\ lookupEnv(env,\ x)
\hspace{2cm}
v\ =\ lookupCellValue(m,\ a)
\\ \hline
(v,\ m)\ =\ eval(x,\ env,\ m)
\end{array} EvalIdentReference $$
<br /><br /><br />
$$\begin{array} &
v\ =\ lookupEnv(env,\ x)
\hspace{2cm}
v\ \neq a
\\ \hline
(v,\ m)\ =\ eval(x,\ env,\ m)
\end{array} EvalIdentNonReference $$


#### AssignVar Inference Rules
<br />
$$\begin{array} &
a\ =\ lookupEnv(env,\ x)
\hspace{2cm}
(v,\ mp)\ =\ eval(e,\ env,\ m)
\hspace{2cm}
mpp\ = assignToCell(mp,\ a,\ v)
\\ \hline
(v,\ mpp)\ =\ eval(AssignVar(x,\ e),\ env,\ m)
\end{array} EvalAssignVarOK$$
<br /><br /><br />
$$\begin{array} &
v\ =\ lookupEnv(env,\ x)
\hspace{2cm}
v\ \neq a
\\ \hline
ERROR\ =\ eval(AssignVar(x,\ e),\ env,\ m)
\end{array} EvalAssignVarError$$

#### LetVar Inference Rule
<br />
$$\begin{array} &
(v_{bind},\ mp)\ =\ eval(e_{bind},\ env,\ m)
\hspace{1cm}
(mpp,\ a)\ =\ createNewCell(mp,\ v_{bind})
\hspace{1cm}
(v_{body},\ mppp)\ = eval(e_{body},\ env\ +\ {x\ ->\ a},\ mpp)
\\ \hline
(v_{body},\ mppp)\ =\ eval(let\ var\ x\ =\ e_{bind}\ in\ e_{body},\ env,\ m)
\end{array} EvalLetVarOK$$

### Interpreter Code
* basically the same but a few different nodes with special behavior
* Ident(Identifier) is somewhat special
    * it might be a simple value in the environment
    * it might be a reference
    * Our code will have to check and behave accordingly

In [5]:
sealed trait Expr
sealed trait Value extends Expr
type Identifier = String
type Address = Int
type Number = Double

case class Ident(x:Identifier) extends Expr
case class Let(x:Identifier, eBind:Expr, eBody:Expr) extends Expr
case class LetVar(x:Identifier, eBind:Expr, eBody:Expr) extends Expr
case class AssignVar(x:Identifier, eRight:Expr) extends Expr
case class Plus(e1:Expr, e2:Expr) extends Expr

case class Reference(a:Address) extends Value
case class Const(n:Number) extends Value


sealed trait Environment 
case object EmptyEnv extends Environment
case class Extend(x: Identifier, v: Value, sigma: Environment) extends Environment
case class ExtendRec(f: Identifier, x: Identifier, e: Expr, sigma: Environment ) extends Environment

def lookupEnv(env: Environment, x: Identifier): Value = env match {
    case EmptyEnv => throw new IllegalArgumentException(s"Error could not find string $x in environment")
    case Extend(y, v, _) if y == x => v
    case Extend(_, _, envp) => lookupEnv(envp, x)
    case _ => ???
}
 

case class ImmutableStore(val nCells: Int, val storeMap: Map[Address, Value])

def createNewCell(s: ImmutableStore, v: Value): (ImmutableStore, Address) = {
    /*- make a new cell -*/
    val j = s.nCells
    val nMap = s.storeMap + (j -> v)
    val nStore = ImmutableStore(s.nCells + 1, nMap) // Make a new store with one more cell
    (nStore, j)
}    

def lookupCellValue(s: ImmutableStore, j: Address): Value = {
    if (s.storeMap.contains(j)){
        s.storeMap(j)
    } else {
        throw new IllegalArgumentException(s"Illegal lookup of nonexistant location $j")
    }
}

def assignToCell(s: ImmutableStore, j: Address, v: Value): ImmutableStore = {
    if (s.storeMap.contains(j)){
        val nMap = s.storeMap + (j -> v) // Update the store map.
        ImmutableStore(s.nCells, nMap)
    } else {
        throw new IllegalArgumentException(s"Illegal assignment to nonexistent location $j")
    }
}


def eval(e:Expr, env:Environment = EmptyEnv, m:ImmutableStore = ImmutableStore(0, Map())): (Value, ImmutableStore) = {
    e match {
        
        case v:Value => (v, m)
        
        case Plus(e1,e2) => eval(e1,env,m) match {
            case (Const(n1),mp) => eval(e2,env,mp) match {
                case (Const(n2),mpp) => (Const(n1+n2),mpp)
                case _ => throw new IllegalArgumentException("e2 not a number")
            }
            case _ => throw new IllegalArgumentException("e1 not a number")
        }
        
        case Let(x,eBind,eBody) => {
            val (vBind, mp) = eval(eBind,env,m)
            val envp = Extend(x, vBind, env)
            eval(eBody,envp,mp)
        }
        
        
        
        case AssignVar(x,eRight) => {
            lookupEnv(env, x) match {
                case Reference(a) => {
                    val (vRight, mp) = eval(eRight,env,m)
                    val mpp = assignToCell(mp, a, vRight)
                    (vRight, mpp)
                }
                case _ => println("foo"); ???
            }
        }
        
        case Ident(x) => {
            val v = lookupEnv(env,x) match {
                case Reference(a) => lookupCellValue(m, a)
                case v => v
            }
            (v,m)
        }
        
        case LetVar(x,eBind,eBody) => {
            val (vBind, mp) = eval(eBind,env,m)
            val (mpp, aBind) = createNewCell(mp, vBind)
            val envp = Extend(x, Reference(aBind), env)
            eval(eBody, envp, mpp)
        }
        
    }
}




// TESTS

// CONCRETE LETTUCE
/*
    let var x = 10 in
    let y = x + 1 in
    let z = AssignVar(x,y) in
    x
*/
// ABSTRACT LETTUCE
val e0 = LetVar(
    "x",
    Const(10.0),
    Let(
        "y",
        Plus(Ident("x"),Const(1.0)),
        Let(
            "z",
            AssignVar("x",Ident("y")),
            Ident("x")
        )
    )
)

val v0Expected = Const(11.0)
val (v0Found, _) = eval(e0)
if (v0Expected == v0Found) {
    println("WOOT WOOT!")
} else {
    println("FAILED:")
    println(s"\tEVALUATING: $e0")
    println(s"\tEXPECTED : $v0Expected")
    println(s"\tFOUND    : $v0Found")
}

WOOT WOOT!


defined trait Expr
defined trait Value
defined type Identifier
defined type Address
defined type Number
defined class Ident
defined class Let
defined class LetVar
defined class AssignVar
defined class Plus
defined class Reference
defined class Const
defined trait Environment
defined object EmptyEnv
defined class Extend
defined class ExtendRec
defined function lookupEnv
defined class ImmutableStore
defined function createNewCell
defined function lookupCellValue
defined function assignToCell
defined function eval
e0: LetVar = LetVar(
  "x",
  Const(10.0),
  Let(
    "y",
    Plus(Ident("x"), Const(1.0)),
    Let("z", AssignVar("x", Ident("y")), Ident("x"))
  )
)
v0Expected: Const = Const(11.0)
v0Found: Value = Const(11.0)

## CPS Intro
* We'll look at this deeper on Thursday and through next week
* But I want to give you some introduction to CPS
* And jump ahead to one of the places it becomes quite useful (short circuiting)

### Factorial

* We looked at simple recursion:

In [19]:
def fact(x:Int):Int = {
    require (x >= 0)
    if (x == 0) 1
    else x * fact(x-1)
}

defined function fact

* We looked at tail recursion:

In [20]:
def factTail(x:Int):Int = {
    require (x >= 0)
    def help(x:Int, acc:Int):Int = {
        if (x == 0) acc
        else help(x-1, acc * x)
    }
    help(x, 1)
}

defined function factTail

* Now it's time to look at tail recursion using the Continuation Passing Style! (or CPS)
* Consider the definition of factCPS provided below
* does it work?
* is it tail recursive?
* Is it useful?

In [25]:
def factCPS(x:Int):Int = {
    require(x >= 0)
    def loop(x:Int)(andThen:Int => Int):Int = {
        if (x == 0) andThen(1)
        else loop(x-1){ factOfXM1 => andThen(factOfXM1 * x) }
    }
    loop(x){ factOfX => factOfX }
}

defined function factCPS

#### Testing Factorial

In [24]:
def fact(x:Int):Int = {
    require (x >= 0)
    if (x == 0) 1
    else x * fact(x-1)
}

def factTail(x:Int):Int = {
    require (x >= 0)
    def help(x:Int, acc:Int):Int = {
        if (x == 0) acc
        else help(x-1, acc * x)
    }
    help(x, 1)
}

def factCPS(x:Int):Int = {
    require(x >= 0)
    def loop(x:Int)(andThen:Int => Int):Int = {
        if (x == 0) andThen(1)
        else loop(x-1){ acc => andThen(acc * x) }
    }
    loop(x)(r => r)
}

def testAFact(f:Int => Int, x:Int, expected:Int):Unit = {
    val got = f(x)
    if (expected == got) {
        println("PASSED") 
    } else {
        println(s"FAILED $f($x)")
        println(s"EXPECTED : $expected")
        println(s"GOT      : $got")
    }
}

def testEachFact(x:Int, expected:Int):Unit = {
    List(fact(_), factTail(_), factCPS(_)) map { (f) => testAFact(f,x,expected) }
}

testEachFact(3, 6)
testEachFact(4, 24)

PASSED
PASSED
PASSED
PASSED
PASSED
PASSED


defined function fact
defined function factTail
defined function factCPS
defined function testAFact
defined function testEachFact

### Usefulness
* this can be useful....
* What is the product of each element in List(0,20,80,2212312,24351,34123093,9)?
* Some problems have obvious solutions - that require little computation to solve
* we can short curcuit tail recursion using CPS

In [6]:
// straight forward
// easy to read
// not exactly optimized
def multList(l:List[Int]):Int = l reduce {_*_}


// harder to read
// optimized agains 0s
// scan the list once
// return 0 if there are any 0s in the list (without performing any multiplication)
// return the product of the list's elements so long as there are no 0s in the list
def multListCPS(l:List[Int]):Int = {
    require(l != Nil)
    def help(l:List[Int])(andThen:Int => Int):Int = l match {
        case Nil => andThen(1)
        case 0::_ => 0  // never call andThen!!!!
        case h::t => help(t){ tProduct => andThen(tProduct * h) }
    }
    help(l)(r => r)
}

defined function multList
defined function multListCPS

#### Non-short curcuit table example
* consider loop(5::6::Nil)(r => r)
<table>
    <caption>loop(5::6::Nil)(r => r)</caption>
    <tr>
        <th>i</th>
        <th>l_i</th>
        <th>andThen_i</th>
        <th>notes_i</th>
        <th>loop(l_i,andThen_i)</th>
    </tr>
    <tr>
        <td>0</td><!-- i -->
        <td>5::6::Nil</td><!-- l_i -->
        <td>r => r</td><!-- andThen_i -->
        <td>
            <ul>
                <li>hit 3rd case</li>
                <li>h = 5</li>
                <li>t = 6::Nil</li>
                <li>loop(6::Nil){ tProduct => andThen_0(tProduct \* 5) }</li>
            </ul>
        </td><!-- notes_i -->
        <td>30</td><!-- loop(l_i,andThen_i) -->
    </tr>
        <tr>
        <td>1</td><!-- i -->
        <td>6::Nil</td><!-- l_i -->
        <td>tProduct => andThen_0(tProduct \* 5)</td><!-- andThen_i -->
        <td>
            <ul>
                <li>hit 3rd case</li>
                <li>h = 6</li>
                <li>t = Nil</li>
                <li>loop(Nil){ tProduct => andThen_1(tProduct \* 6) }</li>
            </ul>
        </td><!-- notes_i -->
        <td>30</td><!-- loop(l_i,andThen_i) -->
    </tr>
    <tr>
        <td>2</td><!-- i -->
        <td>Nil</td><!-- l_i -->
        <td>tProduct => andThen_1(tProduct \* 6)</td><!-- andThen_i -->
        <td>
            <ul>
                <li>hit 1st case</li>
                <li>andThen_2(1)</li>
                <li>tProduct => andThen_1(tProduct \* 6)(1)</li>
                <li>andThen_1(1 \* 6)</li>
                <li>andThen_1(6)</li>
                <li>tProduct => andThen_0(tProduct \* 5)(6)</li>
                <li>andThen_0(6 \* 5)</li>
                <li>andThen_0(30)</li>
                <li>r => r(30)</li>
                <li>30</li>
            </ul>
        </td><!-- notes_i -->
        <td>30</td><!-- loop(l_i,andThen_i) -->
    </tr>
</table>

#### Short curcuit table example
* consider loop(5::0::6::Nil)(r => r)
<table>
    <caption>loop(5::0::6::Nil)(r => r)</caption>
    <tr>
        <th>i</th>
        <th>l_i</th>
        <th>andThen_i</th>
        <th>notes_i</th>
        <th>loop(l_i,andThen_i)</th>
    </tr>
    <tr>
        <td>0</td><!-- i -->
        <td>5::0::6::Nil</td><!-- l_i -->
        <td>r => r</td><!-- andThen_i -->
        <td>
            <ul>
                <li>hit 3rd case</li>
                <li>h = 5</li>
                <li>t = 0::6::Nil</li>
                <li>loop(0::6::Nil){ tProduct => andThen_0(tProduct \* 5) }</li>
            </ul>
        </td><!-- notes_i -->
        <td>0</td><!-- loop(l_i,andThen_i) -->
    </tr>
    <tr>
        <td>1</td><!-- i -->
        <td>0::6::Nil</td><!-- l_i -->
        <td>tProduct => andThen_0(tProduct \* 5)</td><!-- andThen_i -->
        <td>
            <ul>
                <li>hit 2nd case</li>
                <li>do not call andThen</li>
                <li>return 0</li>
            </ul>
        </td><!-- notes_i -->
        <td>0</td><!-- loop(l_i,andThen_i) -->
    </tr>
</table>

## Solutions
* Please don't look at these until after class

### Interpreter

In [12]:
sealed trait Expr
sealed trait Value extends Expr
type Identifier = String
type Address = Int
type Number = Double

case class Ident(x:Identifier) extends Expr
case class Let(x:Identifier, eBind:Expr, eBody:Expr) extends Expr
case class LetVar(x:Identifier, eBind:Expr, eBody:Expr) extends Expr
case class AssignVar(x:Identifier, eRight:Expr) extends Expr
case class Plus(e1:Expr, e2:Expr) extends Expr

case class Reference(a:Address) extends Value
case class Const(n:Number) extends Value


sealed trait Environment 
case object EmptyEnv extends Environment
case class Extend(x: Identifier, v: Value, sigma: Environment) extends Environment
case class ExtendRec(f: Identifier, x: Identifier, e: Expr, sigma: Environment ) extends Environment

case class ImmutableStore(val nCells: Int, val storeMap: Map[Address, Value])

def lookupEnv(env: Environment, x: Identifier): Value = env match {
    case EmptyEnv => throw new IllegalArgumentException(s"Error could not find string $x in environment")
    case Extend(y, v, _) if y == x => v
    case Extend(_, _, envp) => lookupEnv(envp, x)
    case _ => ???
}
  

def createNewCell(s: ImmutableStore, v: Value): (ImmutableStore, Address) = {
    /*- make a new cell -*/
    val j = s.nCells
    val nMap = s.storeMap + (j -> v)
    val nStore = ImmutableStore(s.nCells + 1, nMap) // Make a new store with one more cell
    (nStore, j)
}    

def lookupCellValue(s: ImmutableStore, j: Address): Value = {
    if (s.storeMap.contains(j)){
        s.storeMap(j)
    } else {
        throw new IllegalArgumentException(s"Illegal lookup of nonexistant location $j")
    }
}

def assignToCell(s: ImmutableStore, j: Address, v: Value): ImmutableStore = {
    if (s.storeMap.contains(j)){
        val nMap = s.storeMap + (j -> v) // Update the store map.
        ImmutableStore(s.nCells, nMap)
    } else {
        throw new IllegalArgumentException(s"Illegal assignment to nonexistent location $j")
    }
}

def eval(e:Expr, env:Environment = EmptyEnv, m:ImmutableStore = ImmutableStore(0, Map())): (Value, ImmutableStore) = {
    e match {
        case v:Value => (v, m)
        case Plus(e1,e2) => eval(e1,env,m) match {
            case (Const(n1),mp) => eval(e2,env,mp) match {
                case (Const(n2),mpp) => (Const(n1+n2),mpp)
                case _ => throw new IllegalArgumentException("e2 not a number")
            }
            case _ => throw new IllegalArgumentException("e1 not a number")
        }
        case AssignVar(x,eRight) => lookupEnv(env,x) match {
            case Reference(a) => {
                val (vRight, mp) = eval(eRight,env,m)
                val mpp = assignToCell(mp,a,vRight)
                (vRight, mpp)
            }
            case _ => throw new IllegalArgumentException("x not a reference")
        }
        case Ident(x) => lookupEnv(env,x) match {
            case Reference(a) => (lookupCellValue(m, a), m)
            case v => (v, m)
        }
        case Let(x,eBind,eBody) => {
            val (vBind, mp) = eval(eBind,env,m)
            val envp = Extend(x, vBind, env)
            eval(eBody,envp,mp)
        }
        case LetVar(x,eBind,eBody) => {
            val (vBind, mp) = eval(eBind,env,m)
            val (mpp, a) = createNewCell(mp, vBind)
            val envp = Extend(x, Reference(a), env)
            eval(eBody,envp,mpp)
        }
    }
}

// CONCRETE LETTUCE
/*
    let var x = 10 in
    let y = x + 1 in
    let z = AssignVar(x,y) in
    x
*/
// ABSTRACT LETTUCE
val e0 = LetVar(
    "x",
    Const(10.0),
    Let(
        "y",
        Plus(Ident("x"),Const(1.0)),
        Let(
            "z",
            AssignVar("x",Ident("y")),
            Ident("x")
        )
    )
)

val v0Expected = Const(11.0)
val (v0Found, _) = eval(e0)
if (v0Expected == v0Found) {
    println("WOOT WOOT!")
} else {
    println("FAILED:")
    println(s"\tEVALUATING: $e0")
    println(s"\tEXPECTED : $v0Expected")
    println(s"\tFOUND : $v0Found")
}

WOOT WOOT!


defined trait Expr
defined trait Value
defined type Identifier
defined type Address
defined type Number
defined class Ident
defined class Let
defined class LetVar
defined class AssignVar
defined class Plus
defined class Reference
defined class Const
defined trait Environment
defined object EmptyEnv
defined class Extend
defined class ExtendRec
defined class ImmutableStore
defined function lookupEnv
defined function createNewCell
defined function lookupCellValue
defined function assignToCell
defined function eval
e0: LetVar = LetVar(
  "x",
  Const(10.0),
  Let(
    "y",
    Plus(Ident("x"), Const(1.0)),
    Let("z", AssignVar("x", Ident("y")), Ident("x"))
  )
)
v0Expected: Const = Const(11.0)
v0Found: Value = Const(11.0)

## Overview
* Implicit References
* Eval with Implicit References
* CPS Intro

## TODOs:
* Homework and Quiz 6 are due Friday 
* Project 2 live?
    * It isn't due until after Spring Break
    * It will create Lettuce with Figs
    * Or rather, a variant of lettuce that works with Figures
        * lines
        * circles
        * squares
        * and so on
    * it can be used to create neat fractle patterns
    * with a few extentions it could be used to create a tool similar to power point
* Professional Community Week at the EC
    * lots of events at the EC this week
    * see Piazza for more details
* Spot Exam 3 is now April 5 (after Spring Break! Woo!!)